[Cook County Assessors Office Code](https://gitlab.com/ccao-data-science---modeling)

Data Sources:

* [Cook County GIS Open Data](https://hub-cookcountyil.opendata.arcgis.com/)

In [1]:
# import packages
import csv, datetime, glob, joblib, math, pickle, pydot, time, os, sklearn

from dask.distributed import Client, progress
from datetime import datetime
from IPython.display import display
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.tree import export_graphviz
from tqdm import tqdm

In [2]:
dataset_dropdown = widgets.Dropdown(
    options = glob.glob("./data/processed/*.pkl"),
    value = "./data/processed/Cleaned_Chicago_Sales.pkl",
    description = "Data: "
)
display(dataset_dropdown)

Dropdown(description='Data: ', index=2, options=('./data/processed/Cleaned_Chicago_Sales_wACS.pkl', './data/pr…

In [3]:
# Data from: https://datacatalog.cookcountyil.gov/Property-Taxation/Cook-County-Assessor-s-Residential-Sales-Data/5pge-nu6u
dataset = dataset_dropdown.value
print("Loading data from {}".format(dataset))
ml_df = pd.read_pickle(dataset)
print("Data frame has {} rows and {} columns".format(len(ml_df), len(ml_df.columns)))
ml_df.head()

Loading data from ./data/processed/Cleaned_Chicago_Sales.pkl
Data frame has 326484 rows and 170 columns


,PIN,Property Class,Neighborhood Code,Land Square Feet,Town Code,Type of Residence,Apartments,Wall Material,Roof Material,Rooms,...,Square of Latitude,Log of Longitude,SQRT of Longitude,Square of Longitude,Log of Lot Size Squared,SQRT of Lot Size Squared,Square of Lot Size Squared,Log of Rooms,SQRT of Rooms,Square of Rooms
2,16094150130000,211,13,4500.0,77,3.0,6.0,2.0,2.0,24.0,...,1754.268799,4.474504,9.367557,7700.257324,16.823666,4500.0,4.100625e+14,3.178054,4.898980,576.0
5,4252000820000,204,100,33898.0,25,5.0,0.0,2.0,4.0,11.0,...,1771.669678,4.474834,9.369101,7705.334961,20.862223,33898.0,1.320372e+18,2.397895,3.316625,121.0
8,14322110150000,208,12,3720.0,74,3.0,0.0,2.0,6.0,9.0,...,1757.492310,4.473360,9.362197,7682.651367,16.442959,3720.0,1.915013e+14,2.197225,3.000000,81.0
9,27021200080000,204,34,16079.0,28,1.0,0.0,3.0,1.0,7.0,...,1733.995605,4.475452,9.371996,7714.864746,19.370539,16079.0,6.683995e+16,1.945910,2.645751,49.0
11,13121080620000,204,42,7560.0,71,1.0,0.0,2.0,1.0,6.0,...,1762.405518,4.473950,9.364961,7691.726074,17.861254,7560.0,3.266534e+15,1.791759,2.449490,36.0


In [4]:
ml_df = pd.read_pickle(dataset)
drop_these = ["PIN", "Sale Date", 
              "Estimate (Land)", "Estimate (Building)", 
              "Deed No.", "Pure Market Filter"]
tree_slider = widgets.SelectionSlider(
    options=[ 2**i for i in range(1,10)],
    value=64,
    description="N Trees",
)
start_date, end_date = min(ml_df["Sale Date"]), max(ml_df["Sale Date"])
dates = pd.date_range(start_date, end_date, freq="D")
dates_slider = widgets.SelectionRangeSlider(
    options = [ (date.strftime("%d %b %Y"), date) for date in dates ],
    index=(0,len(dates)-1),
    layout={'width':'500px'}
)
prices = ml_df["Sale Price"].to_numpy()
price_range = range( math.floor(min(prices)/1000), math.ceil(max(prices)/1000), 1)
price_slider = widgets.SelectionRangeSlider(
    options =  price_range,
    index=(0,len(price_range)-1),
    layout={'width':'500px'}
)
label_transform_dropdown = widgets.Dropdown(
    options=["", "sqrt", "log"],
    description="Transform"
)
display(tree_slider, dates_slider, price_slider, label_transform_dropdown)

SelectionSlider(description='N Trees', index=5, options=(2, 4, 8, 16, 32, 64, 128, 256, 512), value=64)

SelectionRangeSlider(index=(0, 2554), layout=Layout(width='500px'), options=(('02 Jan 2013', Timestamp('2013-0…

SelectionRangeSlider(index=(0, 9389), layout=Layout(width='500px'), options=(10, 11, 12, 13, 14, 15, 16, 17, 1…

Dropdown(description='Transform', options=('', 'sqrt', 'log'), value='')

In [5]:
ml_df = pd.read_pickle(dataset)
date_slider_i = dates_slider.index
start, end = dates[date_slider_i[0]], dates[date_slider_i[1]]
label_transform = label_transform_dropdown.value
#print(start, end)
ml_df = ml_df[ml_df["Sale Date"] <= end]
ml_df = ml_df[ml_df["Sale Date"] >= start]
price_i = price_slider.index
low_price, high_price = price_range[price_i[0]], price_range[price_i[1]]
ml_df = ml_df[ml_df["Sale Price"] >= low_price*1000]
ml_df = ml_df[ml_df["Sale Price"] <= high_price*1000]

ml_df = ml_df.drop(drop_these, axis=1)
print("Data frame has {} rows and {} columns".format(len(ml_df), len(ml_df.columns)))
ml_df.head()

Data frame has 326483 rows and 164 columns


,Property Class,Neighborhood Code,Land Square Feet,Town Code,Type of Residence,Apartments,Wall Material,Roof Material,Rooms,Bedrooms,...,Square of Latitude,Log of Longitude,SQRT of Longitude,Square of Longitude,Log of Lot Size Squared,SQRT of Lot Size Squared,Square of Lot Size Squared,Log of Rooms,SQRT of Rooms,Square of Rooms
2,211,13,4500.0,77,3.0,6.0,2.0,2.0,24.0,12.0,...,1754.268799,4.474504,9.367557,7700.257324,16.823666,4500.0,4.100625e+14,3.178054,4.898980,576.0
5,204,100,33898.0,25,5.0,0.0,2.0,4.0,11.0,4.0,...,1771.669678,4.474834,9.369101,7705.334961,20.862223,33898.0,1.320372e+18,2.397895,3.316625,121.0
8,208,12,3720.0,74,3.0,0.0,2.0,6.0,9.0,5.0,...,1757.492310,4.473360,9.362197,7682.651367,16.442959,3720.0,1.915013e+14,2.197225,3.000000,81.0
9,204,34,16079.0,28,1.0,0.0,3.0,1.0,7.0,3.0,...,1733.995605,4.475452,9.371996,7714.864746,19.370539,16079.0,6.683995e+16,1.945910,2.645751,49.0
11,204,42,7560.0,71,1.0,0.0,2.0,1.0,6.0,5.0,...,1762.405518,4.473950,9.364961,7691.726074,17.861254,7560.0,3.266534e+15,1.791759,2.449490,36.0


# Random Forest

In [6]:
ml_df.describe()

,Property Class,Neighborhood Code,Land Square Feet,Town Code,Type of Residence,Apartments,Wall Material,Roof Material,Rooms,Bedrooms,...,Square of Latitude,Log of Longitude,SQRT of Longitude,Square of Longitude,Log of Lot Size Squared,SQRT of Lot Size Squared,Square of Lot Size Squared,Log of Rooms,SQRT of Rooms,Square of Rooms
count,326483.000000,326483.000000,3.264830e+05,326483.000000,326483.000000,326483.000000,326483.000000,326483.000000,326483.000000,326483.000000,...,326483.000000,326483.000000,326483.000000,326483.000000,326483.000000,3.264830e+05,3.264830e+05,326483.000000,326483.000000,326483.000000
mean,220.004227,108.797316,7.005201e+03,44.663529,2.117905,0.387962,1.947195,1.167032,7.066491,3.536962,...,1751.856323,4.474832,9.369090,7705.333008,17.200993,7.005201e+03,4.316006e+20,1.885620,2.608716,60.998215
std,29.478819,100.022328,1.278797e+04,23.764591,1.281432,1.042608,0.782431,0.611666,3.326103,1.515634,...,14.260287,0.001656,0.007759,25.555960,1.278628,1.278797e+04,inf,0.345431,0.510973,148.063934
min,202.000000,10.000000,1.490000e+02,10.000000,1.000000,-5.000000,1.000000,1.000000,2.000000,1.000000,...,1719.750000,4.471923,9.355474,7660.605469,10.007893,1.490000e+02,4.928844e+08,0.693147,1.414214,4.000000
25%,203.000000,34.000000,3.720000e+03,24.000000,1.000000,0.000000,1.000000,1.000000,5.000000,3.000000,...,1741.469788,4.473735,9.363955,7688.422119,16.442959,3.720000e+03,1.915013e+14,1.609438,2.236068,25.000000
50%,205.000000,80.000000,5.125000e+03,37.000000,2.000000,0.000000,2.000000,1.000000,6.000000,3.000000,...,1753.235474,4.474535,9.367699,7700.726074,17.083771,5.125000e+03,6.898831e+14,1.791759,2.449490,36.000000
75%,211.000000,150.000000,8.000000e+03,71.000000,2.000000,0.000000,2.000000,1.000000,8.000000,4.000000,...,1763.889832,4.475550,9.372454,7716.371826,17.974394,8.000000e+03,4.096000e+15,2.079442,2.828427,64.000000
max,295.000000,600.000000,2.980767e+06,77.000000,8.000000,6.000000,4.000000,6.000000,241.000000,90.000000,...,1776.957886,4.480327,9.394866,7790.446289,29.815382,2.980767e+06,7.894272e+25,5.484797,15.524175,58081.000000


In [7]:
label_cols = ["Sale Price"]

labels = np.array(ml_df[label_cols])
if label_transform == "":
    pass
elif label_transform == "sqrt":
    labels = np.sqrt(labels)
elif label_transform == "log":
    labels = np.log(labels)
features = ml_df.drop(label_cols, axis=1)
feature_list = list(features.columns)
#print(feature_list)
features = np.array(features)

In [8]:
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(np.nan_to_num(features), np.nan_to_num(labels), test_size = 0.25, random_state = 42)
train_labels, test_labels = train_labels.ravel(), test_labels.ravel()
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (244862, 163)
Training Labels Shape: (244862,)
Testing Features Shape: (81621, 163)
Testing Labels Shape: (81621,)


In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
ntrees = tree_slider.value
print("Running RFR with {} trees".format(ntrees))
rf = RandomForestRegressor(n_estimators = ntrees, random_state = 42)
# Train the model on training data
with Client(processes=False, threads_per_worker=5, n_workers=1, memory_limit='10GB') as client:
    with joblib.parallel_backend("dask"):
        rf.fit(train_features, train_labels)
pickle.dump( rf, open( "rfr.pkl", "wb" ) )

Running RFR with 64 trees


In [ ]:
plt.hist([rf.estimators_[i].get_depth() for i in range(ntrees)],density=True)
plt.xlabel("Depth/Height of Tree")
plt.ylabel("Density")
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

In [ ]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
print(label_transform)
if label_transform == "":
    pass
elif label_transform == "sqrt":
    predictions = np.square(predictions)
    train_labels = np.square(train_labels)
    test_labels = np.square(test_labels)
elif label_transform == "log":
    predictions = np.exp(predictions)
    train_labels = np.exp(train_labels)
    test_labels = np.exp(test_labels)
# Calculate the absolute errors
errors = predictions-test_labels
print("Mean Sale Price in Training Set: ${:7.2f}".format(np.mean(train_labels)))
print("Mean Sale Price in Test Set: ${:7.2f}".format(np.mean(test_labels)))
print('Mean Error: ${:7.2f}'.format(np.mean(errors)))
plt.boxplot(errors)
plt.show()

In [ ]:
# Print out the mean absolute error (mae)
mae = np.mean(np.abs(errors))
print('Mean Absolute Error: ${:7.2f}'.format(mae))
plt.boxplot(np.abs(errors))
plt.show()

In [ ]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (np.abs(errors) / test_labels)
# Calculate and display accuracy
mape = np.mean(mape)
print('MAPE: {:3.2f}%'.format(mape))
plt.boxplot(np.abs(errors)/test_labels)
plt.show()

In [ ]:
plt.scatter(test_labels, errors)
plt.title("Price vs. Error")
plt.xlabel("Price")
plt.ylabel("Error")
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

In [ ]:
importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_],axis=0)
indices = np.argsort(importances)[::-1][:20]

# Print the feature ranking
print("Feature ranking:")

for f in range(len(indices)):
    print("{:2d}. feature {:2d} = {:<30s} ({:1.4f})".format(f+1, indices[f], feature_list[indices[f]], importances[indices[f]]))

# Plot the impurity-based feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(len(indices)), importances[indices], color="r", yerr=std[indices], align="center")
plt.xticks(range(len(indices)), [ feature_list[i] for i in indices], rotation=45, ha="right")
plt.xlim([-1, len(indices)])
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

In [ ]:
feature_list = list(feature_list)
index = feature_list.index('Property Class')
classes = set()
for i in range(len(test_features)):
    classes.add(int(test_features[i][index]))
pcerrors = {}
#print(classes)
for elem in classes:
    pcerrors[elem] = {}
    pcerrors[elem]["error"] = []
    pcerrors[elem]["abs_err"] = []
abs_error = np.abs(errors)
for i in range(len(test_features)):
    pc = int(test_features[i][index])
    pcerrors[pc]["error"].append(errors[i])
    pcerrors[pc]["abs_err"].append(abs_error[i])
classes = list(classes)
classes.sort()
for elem in classes:
    print("Mean error for class {} is ${:7.2f} and mean absolute error is ${:7.2f}".format(elem, np.mean(pcerrors[elem]["error"]), np.mean(pcerrors[elem]["abs_err"]) ))
plt.bar([str(i) for i in classes], [np.mean(pcerrors[elem]["error"]) for elem in classes], yerr=[np.std(pcerrors[elem]["error"])/100 for elem in classes])
plt.xlabel("Property Classes", fontsize=20)
plt.ylabel("Mean Error (error bars = 1% $\sigma$)", fontsize=20)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

In [ ]:
plt.bar([str(i) for i in classes], [np.mean(pcerrors[elem]["abs_err"]) for elem in classes], yerr=[np.std(pcerrors[elem]["error"]) for elem in classes])
plt.xlabel("Property Classes", fontsize=20)
plt.ylabel("Mean Abs Error (error bars = $\sigma$)", fontsize=20)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

In [ ]:
to_plt = [ pcerrors[pc]["error"] for pc in classes ]
plt.boxplot(to_plt)
plt.xticks(range(len(classes)+1), [""]+classes)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

In [ ]:
# check if depth is too high
if np.mean([rf.estimators_[0].get_depth() for i in range(ntrees)]) < 8:
    tree = rf.estimators_[0]
    export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)
    (graph, ) = pydot.graph_from_dot_file('tree.dot')
    graph.write_png('tree.png')
else:
    print("Tree height is too large to realistically plot.")